In [36]:
import requests
import json
import os
import time
from datetime import datetime
def reset_embeddings_db(ticket_id):
    url = "http://localhost:11435/api/embeddings/reset_embeddings_db"
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps({
        "embeddings_db":"TicketEmbeddings_"+str(ticket_id)
    }))
reset_embeddings_db(1)

In [41]:

def generate_ticket_info(ticket_id,section,content):
    data = {
        "source":section,
        "content":content,
        "overlap":50,
        "chunk_size":255,
        "check_existing":True,
        "embeddings_db":"TicketEmbeddings_"+str(ticket_id),
    }
    return data
def batch_insert(collection):

    url = "http://localhost:11435/api/embeddings/batch_insert_text_embeddings"
    headers = {
        "Content-Type": "application/json"
    }
    response = requests.post(url, data=json.dumps({'embeddings':collection}), headers=headers)

In [42]:
ticket_information = {
    "ticket_id": 1,
    "ticket_description": "This is a ticket description",
    "ticket_status": "Open",
    "ticket_priority": "High",
    "ticket_owner": "John Doe",
    "ticket_created": "01-01-2021",
    "ticket_updated": "01-01-2021"
}

ticket_embedding_strings = []
for key, value in ticket_information.items():
    ticket_embedding_strings.append(f"{key} is '{value}'")

In [45]:
embeddings_collection = []
for  item in ticket_embedding_strings:
    embeddings_collection.append(generate_ticket_info(ticket_information["ticket_id"],"ticket",item))
batch_insert(embeddings_collection)

In [35]:
#markdown render to console
def html2print(markdown_data):
    import markdown
    from bs4 import BeautifulSoup
    html = markdown.markdown(markdown_data)
    soup = BeautifulSoup(html, "html.parser")
    print(soup.get_text())


def GenerateAiResponse(ticket_id,system_message,query):
    url = "http://localhost:11435/api/rag"
    data = {
        "prompt":query,
        "system_message":system_message,
        "model":"interstellarninja/hermes-2-theta-llama-3-8b",
        "related_count":20,
        "max_tokens":100,
        "embeddings_db":"TicketEmbeddings_"+str(ticket_id),
        "temperature":0.9
    }
    headers = {
        "Content-Type": "application/json"  
    }
    response = requests.post(url, data=json.dumps(data), headers=headers)
    data= response.json()
    html2print(data["message"]["content"])

GenerateAiResponse(ticket_id=1,system_message="""
You are Kathrin, 18 year old female grad student,
you studdied in the US and now you are in Germany working as a service tech.
You are helpful but stay on topic,you are professional, you are not overly friendly, you are not overly formal.
dont reitterate the ticket data unless necessary.
Only Respond to the user, no detail listing, the customer is not interested in the details of the ticket. and is to blame for the issue.
""",
query="""Please respond to ticket""")

KeyError: 'message'